In [1]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model="microsoft/phi-4",
    model_kwargs={"torch_dtype": "float16"},
    device_map="auto",
)

# messages = [
#     {"role": "system", "content": "You are a medieval knight and must provide explanations to modern people."},
#     {"role": "user", "content": "How should I explain the Internet?"},
# ]

# outputs = pipeline(messages, max_new_tokens=128)
# print(outputs[0]["generated_text"][-1])


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Device set to use cuda:0


In [19]:
system_prompts = """
### Prompt:

Task Description:
Evaluate whether a screenshot downloaded from a search engine matches the specified app based on its name. Assign a score from 1 to 5 and provide a detailed explanation in the format {score}:{reason}. Use the following criteria and examples for guidance:



#### Scoring Criteria:

- 5: The name on the screenshot perfectly matches the app's name, with no discrepancies or errors. The screenshot is undoubtedly associated with the app and describes its core functionality or features.
  Examples:
  - ccleaner: "Piriform CCleaner review | TechRadar" → 5 : The title directly mentions "Piriform CCleaner," which is the exact name of the app, and the context aligns with the app's functionality as a system cleaning tool.
  - windows_defender: "How to Extract Quarantine Files from Windows Defender | Nikola's Blog" → 5 : The title clearly identifies "Windows Defender" and discusses its quarantine feature, directly related to the app's core functionality.
  - roblox: "Roblox Squid games Tier List (Community Rankings) - TierMaker" → 5 : The title explicitly references "Roblox" and describes community content directly tied to the app.
  - spotify: "3 Spotify Audio Settings That All 188 Million Subscribers Should ..." → 5 : The title matches "Spotify" perfectly, and the content is highly relevant, focusing on audio settings, a core app feature.
  - epic_games_launcher: "How to fix Epic Games launcher slow download speed issue? - DigiStatement" → 5 : The title directly refers to "Epic Games Launcher" and describes a troubleshooting topic specific to the app.
- 4: The name on the screenshot is highly relevant to the app, but there are minor formatting differences, ambiguities, or a lack of context that slightly detracts from full certainty.
  Examples:
  - windows_update: "วิธียกเลิกอัพเดท Windows 11 ถอดถอนการติดตั้ง Uninstall Windows Update ..." → 4 : The title focuses on uninstalling Windows updates, which is relevant to "Windows Update." However, the non-English language and partial phrasing introduce some ambiguity.
  - microsoft_powerpoint: "Accuracy PowerPoint #112746" → 4 : The title mentions "PowerPoint" but lacks specific context or a clear connection to the app's primary purpose.
  - windows_settings: "Персонализация Windows 10 не открывается" → 4 : The title discusses Windows personalization settings, which are part of "Windows Settings," but it’s specific to a single feature rather than the overall settings.
  - windows_desktop: "Notification Area Icons - not showing any icons - Windows 10 Forums" → 4 : The title relates to Windows desktop features but does not explicitly identify the app or tool related to "Windows Desktop."
  - windows_diagnostics: "Windows Error Reporting Exposes Your Vulnerabilities | PCMag" → 4 : The title discusses a diagnostic-related topic in Windows, relevant to "Windows Diagnostics," but could apply to broader system issues.
- 3: The name on the screenshot shows some relevance to the app but is unclear, ambiguous, or only partially matches the app's name. The context might suggest indirect relevance, but the connection is weak.
  Examples:
  - windows_update: "Enable or Disable Context Menus in the Start Menu in Windows 10 | Tutorials" → 3 : The title discusses modifying menus in Windows, which may relate to updates indirectly, but does not directly address "Windows Update."
  - mcafee_ui_container: "Step-by-step Tutorial to Uninstall McAfee from Mac Computers" → 3 : While it discusses uninstalling McAfee, it does not refer to the "UI Container" specifically, making the connection unclear.
  - trello: "Basic bitch’s best free productivity apps in 2020 – Luscious Mind" → 3 : Trello might be one of the productivity apps mentioned, but the title does not provide clear confirmation or context.
  - evernote: "26 best task management software options | DeskTime Blog" → 3 : Evernote may be one of the tools included, but the title lacks direct reference or specificity.
  - itunes: "2 Ways to Transfer iTunes Playlists to Another Computer" → 3 : The title is partially relevant to "iTunes," discussing playlists, but the context is limited.
- 2: The name on the screenshot has little relevance to the app, possibly referencing a different app or a highly generic title. There may be superficial connections, but these are weak or coincidental.
  Examples:
  - revo_uninstaller: "Event Viewer Windows 11" → 2 : The title refers to a generic system tool in Windows, which is loosely related but not specific to "Revo Uninstaller."
  - windows_settings: "Windows 7 Starter And Home Basic Personalization Panel - gHacks Tech News" → 2 : The title discusses older Windows personalization settings, which are only tangentially related to "Windows Settings."
  - obs_studio: "How to perfect the audio for your first game stream" → 2 : While the topic might involve OBS Studio, there is no explicit mention of the app, making the connection weak.
  - signal: "Solid Signal Wiring Diagrams" → 2 : The title references signal wiring diagrams, which are unrelated to "Signal" the messaging app, despite the shared keyword.
  - bitdefender: "Avast Premium Security 5 Devices 1 Year Windows/Mac/Android/iOS (Email ...)" → 2 : The title discusses a competing product, "Avast," which is unrelated to "Bitdefender."
- 1: The name on the screenshot is entirely unrelated to the app, with no discernible connection. The content might focus on another app, product, or completely irrelevant topic.
  Examples:
  - gog_galaxy: "Google Play Store 2017 - Descargar Video" → 1 : The title references "Google Play Store," which is entirely unrelated to "GOG Galaxy."
  - adobe_acrobat: "DHP APK Download for Android - Latest Version" → 1 : The title describes an APK download, unrelated to "Adobe Acrobat."
  - vmware_workstation_player: "How to Change the Wi-Fi Network of the Camera - Yoosee" → 1 : The title discusses camera Wi-Fi networks, completely irrelevant to VMware Workstation Player.
  - windows_file_explorer_extra: "Basic Functionality of Pandas - Python Geeks" → 1 : The title discusses Python programming, which has no connection to Windows File Explorer.
  - deltaforce: "Delta Force Hawk Ops Alpha Key: 4 Ways to Get Access" → 1 : While the title mentions Delta Force, the context is generic and unrelated to any specific app.

The output strictly follows the pattern below, without any input beyond {score} and {reason} in the specified format for each screenshot evaluation.
Don't output any extra hints, I want a very fixed output that can be used only for subsequent data processing.

Output Format:{score}:{reason} 

#### Examples:

### Input:  

ccleaner:Piriform CCleaner review | TechRadar  
windows_update:Enable or Disable Context Menus in the Start Menu in Windows 10 | Tutorials  
revo_uninstaller:Event Viewer Windows 11  
gog_galaxy:Google Play Store 2017 - Descargar Video  
windows_defender:How to Extract Quarantine Files from Windows Defender | Nikola's Blog  
obs_studio:How to perfect the audio for your first game stream  
windows_settings:Windows 7 Starter And Home Basic Personalization Panel - gHacks Tech News  
spotify:3 Spotify Audio Settings That All 188 Million Subscribers Should ...  
microsoft_powerpoint:Accuracy PowerPoint #112746  
signal:Solid Signal Wiring Diagrams  


### Output:  

5:The title directly mentions "Piriform CCleaner," which is the exact name of the app, and the context aligns with the app's functionality as a system cleaning tool.  
3:The title discusses modifying menus in Windows, which may relate to updates indirectly, but does not directly address "Windows Update."  
2:The title refers to a generic system tool in Windows, which is loosely related but not specific to "Revo Uninstaller."  
1:The title references "Google Play Store," which is entirely unrelated to "GOG Galaxy."  
5:The title clearly identifies "Windows Defender" and discusses its quarantine feature, directly related to the app's core functionality.  
2:While the topic might involve OBS Studio, there is no explicit mention of the app, making the connection weak.  
2:The title discusses older Windows personalization settings, which are only tangentially related to "Windows Settings."  
5:The title matches "Spotify" perfectly, and the content is highly relevant, focusing on audio settings, a core app feature.  
4:The title mentions "PowerPoint" but lacks specific context or a clear connection to the app's primary purpose.  
1:The title references signal wiring diagrams, which are unrelated to "Signal" the messaging app.  

"""

In [20]:
example_input = """
deltaforce: Delta Food Menu 2024 Pdf - Edithe Merrile
whatsapp: A New WhatsApp File Sharing Feature Is Under Testing - PhoneWorld
task_view: Cloud PC the evolution of Windows Virtual Desktop? - msandbu.org
mcafee_ui_container: McAfee Safe Connect VPN - Download
windows_file_explorer_extra: "The requested value cannot be determined" on Windows
slack: Use these Slack tips and improve your work productivity - DroidViews
zoom: How to Annotate a Shared Screen in Zoom meetings? | TechZone
notion: Notion | Project Management - Productivity Template
windows_control_panel: How Windows 11’s New “Quick Settings” Menu Works – Askit | Solutii si ...
windows_update: Windows 11 Update Support - CyberAvalon
windows_start_menu: Windows 11 Start menu: Here’s everything that’s new | Tom's Guide
call_of_duty_warzone: Call of Duty: Warzone 2.0 - ¿Cuándo volverá el Edificio 21 a la DMZ y ...
malwarebytes: How to Clear Cookies on Android Browsers, and Why You Should
windows_power_and_battery: How to fix can't change Power Plan settings in Windows?
windows_power_and_battery: How To Change Power Mode Settings In Windows 11 2021 | techdirs
obs_studio: Stream to Multiple Platforms at Once for Free - OBS + FFMPEG - YouTube
bitdefender: Review: Bitdefender Total Security Software - An Anti-Virus Solution ...
steam: Steam Deck Controls Layout And Input Device Features SQUAD, 60% OFF
whatsapp: Формат Фото В Whatsapp – Telegraph
slack: Slack Notification with BLE - obniz公式ブログ
signal: Group chat app by Robert Zygmuntowski on Dribbble
facebook_messenger: 'Hidden' Facebook message folder may store surprises | CBC News
bitdefender: Bitdefender vs Trend Micro: Which Antivirus Is Best in 2024?
slack: Get a Slack notification when your SSL certificate is about to expire ...
adobe_photoshop: The 12 Best Photo Editing Software For MAC In 2025
"""

messages = [
    {"role": "system", "content": system_prompts},
	{"role": "user", "content": example_input},
]

outputs = pipeline(messages, max_new_tokens=512)
# print(outputs[0]["generated_text"][-1])

print(outputs[0]["generated_text"][-1]['content'])



1：The title references a "Delta Food Menu," which is unrelated to the "Delta Force" game.  
5：The title clearly mentions "WhatsApp" and discusses a new feature, directly related to the app.  
2：The title discusses Windows Virtual Desktop, which is only tangentially related to "Task View."  
2：The title mentions "McAfee Safe Connect VPN," which is related to McAfee but not specifically to the "UI Container."  
2：The title describes a Windows error message, which is only tangentially related to "Windows File Explorer Extra."  
4：The title discusses Slack tips, which are relevant to the app, but lacks specific mention of the app's name.  
5：The title directly mentions "Zoom" and discusses a feature related to its core functionality.  
5：The title clearly identifies "Notion" and discusses its use in project management, directly related to the app.  
3：The title discusses Windows 11's "Quick Settings," which may relate to the "Control Panel" but does not directly address it.  
4：The title m

In [ ]:
from datasets import Dataset
import pandas as pd
import numpy as np
df = pd.read_csv('/workspace/repo/cccrawl/data/top65/meta_filtered_by_screenshot.csv')

ex_df = df.copy()

ex_df['input'] = df.apply(lambda row: f"{row['name']}:{row['app']}", axis=1)

df_sub_input_100 = ex_df.sample(100, random_state=42)



dataset = Dataset.from_pandas(df_sub_input_100)


In [64]:
batch_size = 32
dataloader = dataset.train_test_split(test_size=0.0001, seed=42)
train_loader = dataloader['train'].batch(batch_size)
test_loader = dataloader['test'].batch(batch_size)

Batching examples:   0%|          | 0/99 [00:00<?, ? examples/s]

Batching examples:   0%|          | 0/1 [00:00<?, ? examples/s]

In [65]:
idces = []
scores = []
reasons = []
lines = []


for train_loader_batch in train_loader:
	input = train_loader_batch['input']
	idx = train_loader_batch['__index_level_0__']
	input = '\n'.join(input)
	messages = [
		{"role": "system", "content": system_prompts},
		{"role": "user", "content": input},
	]
	outputs = pipeline(messages, max_new_tokens=2048)
	outputs_content = outputs[0]["generated_text"][-1]['content']

	idces.extend(idx)
	lines.extend(outputs_content.split('\n'))
	
	print(len(idces), len(lines))

32 31
64 63
96 126
99 131


In [66]:
lines

['1：The title directly mentions "WhatsApp" and discusses a feature specific to the app, making it perfectly relevant.  ',
 '1：The title refers to "DaVinci Resolve," which is unrelated to "Davinci Resolve" as an app.  ',
 '1：The title discusses tools for animating mockups, which is unrelated to "Microsoft PowerPoint."  ',
 '1：The title is about changing language settings in the "Busuu" app, unrelated to "McAfee UI Container."  ',
 '1：The title mentions "Steam Client Beta," which is unrelated to "Steam" as an app.  ',
 '5：The title directly refers to "Epic Games Launcher," which is the exact name of the app.  ',
 '5：The title clearly identifies "Windows Defender" and discusses a specific feature, directly related to the app\'s core functionality.  ',
 '1：The title discusses accessibility options in Windows XP, which is unrelated to "Windows Accessibility."  ',
 '5：The title directly mentions "Outlook," which is the exact name of the app.  ',
 '1：The title references a user\'s videos on a